# 训练模型

## 引入第三方包

In [1]:
from PIL import Image
from keras import backend as K
from keras.utils.vis_utils import plot_model
from keras.models import *
from keras.layers import *

import glob
import pickle

import numpy as np
import tensorflow.gfile as gfile
import matplotlib.pyplot as plt

D:\python\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 定义超参数和字符集

In [2]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
LOWERCASE = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']
UPPERCASE = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
           'V', 'W', 'X', 'Y', 'Z']

CAPTCHA_CHARSET = NUMBER   # 验证码字符集
CAPTCHA_LEN = 4            # 验证码长度
CAPTCHA_HEIGHT = 60        # 验证码高度
CAPTCHA_WIDTH = 160        # 验证码宽度

TRAIN_DATA_DIR = './train-data/' # 验证码数据集目录
TEST_DATA_DIR = './test-data/'

# 每一批训练图片的个数
BATCH_SIZE = 100
# 训练的轮数，即训练第几轮结束
EPOCHS = 2
# 优化器使用adam，其有自适应参数的功能
OPT = 'adam'
# 损失函数
LOSS = 'binary_crossentropy'

# model 是 keras训练出来的模型
# 存放model的目录
MODEL_DIR = './model/train_demo/'
# 文件的后缀名
MODEL_FORMAT = '.h5'
# 训练记录的文件目录，可以查看准确率，loss等参数变化
HISTORY_DIR = './history/train_demo/'
# 文件的后缀名
HISTORY_FORMAT = '.history'

# 定义模型文件和训练记录文件的格式化字符串 {模型目录}captcha_{模型优化器}_{使用的损失函数}_bs_{BATCH_SIZE是多少}_epochs_{EPOCHS轮数}{后缀名}"
filename_str = "{}captcha_{}_{}_bs_{}_epochs_{}{}"

# 定义模型网络结构文件 文件名为captcha_classfication.png
MODEL_VIS_FILE = 'captcha_classfication' + '.png'
# 模型文件
MODEL_FILE = filename_str.format(MODEL_DIR, OPT, LOSS, str(BATCH_SIZE), str(EPOCHS), MODEL_FORMAT)
# 训练记录文件
HISTORY_FILE = filename_str.format(HISTORY_DIR, OPT, LOSS, str(BATCH_SIZE), str(EPOCHS), HISTORY_FORMAT)

## 将 RGB 验证码图像转为灰度图

In [3]:
def rgb2gray(img):
    # Y' = 0.299 R + 0.587 G + 0.114 B 
    # https://en.wikipedia.org/wiki/Grayscale#Converting_color_to_grayscale
    return np.dot(img[...,:3], [0.299, 0.587, 0.114])

## 对验证码中每个字符进行 one-hot 编码

In [4]:
def text2vec(text, length=CAPTCHA_LEN, charset=CAPTCHA_CHARSET):
    print(text)
    text_len = len(text)
    # 验证码长度校验
    if text_len != length:
        raise ValueError('Error: length of captcha should be {}, but got {}'.format(length, text_len))
    
    # 生成一个形如（CAPTCHA_LEN*CAPTHA_CHARSET,) 的一维向量
    # 例如，4个纯数字的验证码生成形如(4*10,)的一维向量
    vec = np.zeros(length * len(charset))
    for i in range(length):
        # One-hot 编码验证码中的每个数字
        # 每个字符的热码 = 索引 + 偏移量
        vec[charset.index(text[i]) + i*len(charset)] = 1
    print('vec',vec)
    return vec

## 将验证码向量解码为对应字符

In [5]:
def vec2text(vector):
    if not isinstance(vector, np.ndarray):
        vector = np.asarray(vector)
    vector = np.reshape(vector, [CAPTCHA_LEN, -1])
    text = ''
    for item in vector:
        text += CAPTCHA_CHARSET[np.argmax(item)]
    return text

## 适配 Keras 图像数据格式

In [6]:
def fit_keras_channels(batch, rows=CAPTCHA_HEIGHT, cols=CAPTCHA_WIDTH):
    if K.image_data_format() == 'channels_first':
        batch = batch.reshape(batch.shape[0], 1, rows, cols)
        input_shape = (1, rows, cols)
    else:
        batch = batch.reshape(batch.shape[0], rows, cols, 1)
        input_shape = (rows, cols, 1)
    
    return batch, input_shape

## 读取训练集

In [18]:
X_train = []
Y_train = []
np.set_printoptions(threshold=np.inf)
# 使用glob方法读取所有png结尾的文件
for filename in glob.glob(TRAIN_DATA_DIR + '*.png'):
    # 将图片打开转换成ndarray格式保存到X_train数组中
    # print(Image.open(filename))
    X_train.append(np.array(Image.open(filename)))
#     print(X_train)    
#     print(np.shape(X_train))
#     break
    # Y_train为读取文件标签，不要后缀名也不要父目录路径
    filename=filename.lstrip(TRAIN_DATA_DIR).rstrip('.png')
    Y_train.append(filename.replace('\\',''))
#     Y_train.append(filename.lstrip(TRAIN_DATA_DIR).rstrip('.png'))

In [19]:
print(Y_train[2])
print(np.shape(X_train))
print(np.shape(Y_train))

0012
(1211, 60, 160, 3)
(1211,)


## 处理训练集图像

In [20]:
# list -> rgb(numpy) 将list转化为numpy数组
X_train = np.array(X_train, dtype=np.float32)
# rgb -> gray 将RGB转成灰度图
X_train = rgb2gray(X_train)
# normalize 规范化
X_train = X_train / 255

# Fit keras channels  keras的适配
X_train, input_shape = fit_keras_channels(X_train)

# 打印X_train的形状和类型，因为可能会生成相同的文件名，所以有的文件会被覆盖，总的文件数量小于上面指定生成的文件数量
print(X_train.shape, type(X_train))
print(input_shape)

(1211, 60, 160, 1) <class 'numpy.ndarray'>
(60, 160, 1)


## 处理训练集标签

In [21]:
Y_train = list(Y_train)

# 对验证码中每个字符按照标签进行 one-hot 编码
for i in range(len(Y_train)):
    Y_train[i] = text2vec(Y_train[i])

# 设置numpy数组
Y_train = np.asarray(Y_train)
# 打印图片形状
print(Y_train.shape, type(Y_train))

0008
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
0010
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0012
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
0013
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
0015
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
0022
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
0023
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
0027
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
0925
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
0926
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
0927
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
0928
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
0934
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
0940
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0942
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.

vec [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
1531
vec [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
1536
vec [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
1544
vec [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
1546
vec [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
1548
vec [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
1555
vec [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
1557
vec [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.

 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
2366
vec [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
2367
vec [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
2372
vec [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
2374
vec [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
2375
vec [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
2376
vec [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
2377
vec [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
2378
vec [0. 0. 1. 0. 0. 0

## 读取测试集，处理对应图像和标签

In [22]:
X_test = []
Y_test = []
for filename in glob.glob(TEST_DATA_DIR + '*.png'):
    X_test.append(np.array(Image.open(filename)))
    filename=filename.lstrip(TEST_DATA_DIR).rstrip('.png')
    Y_test.append(filename.replace('\\',''))
#     Y_test.append(filename.lstrip(TEST_DATA_DIR).rstrip('.png'))

# list -> rgb -> gray -> normalization -> fit keras  分别调用上面定义的函数
X_test = np.array(X_test, dtype=np.float32)
# 灰度处理
X_test = rgb2gray(X_test)
# normalization处理
X_test = X_test / 255
# keras适配
X_test, _ = fit_keras_channels(X_test)

Y_test = list(Y_test)
for i in range(len(Y_test)):
    print(i)
    Y_test[i] = text2vec(Y_test[i])

Y_test = np.asarray(Y_test)

print(X_test.shape, type(X_test))
print(Y_test.shape, type(Y_test))

0
0014
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
1
0033
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
2
0047
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
3
0061
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
4
0065
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
5
0091
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
6
0093
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
7
0101
vec [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.

 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
306
3200
vec [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
307
3201
vec [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
308
3215
vec [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
309
3222
vec [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
310
3225
vec [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
311
3245
vec [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
312
3262
vec [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.

vec [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
550
5884
vec [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
551
5897
vec [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
552
5899
vec [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
553
5922
vec [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
554
5936
vec [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
555
5942
vec [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
556
5946
vec [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0

vec [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
800
8234
vec [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
801
8238
vec [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
802
8255
vec [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
803
8256
vec [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
804
8270
vec [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
805
8276
vec [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
806
8292
vec [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0

## 创建验证码识别模型

In [23]:
# 输入层
inputs = Input(shape = input_shape, name = "inputs")

# 第1层卷积 ,定义了32个卷积核，卷积核的形状为3x3，同时命名为conv1，输入就是我们的输入层的数据
conv1 = Conv2D(32, (3, 3), name = "conv1")(inputs)
# 经过relu激活后得到relu1
relu1 = Activation('relu', name="relu1")(conv1)

# 第2层卷积，将激活值输入到第二层卷积中
conv2 = Conv2D(32, (3, 3), name = "conv2")(relu1)
relu2 = Activation('relu', name="relu2")(conv2)
# 做MaxPooling 设置池化大小为2*2，表示每一个2*2的像素都会变成一个像素，这样整个图像的参数就会变少
pool2 = MaxPooling2D(pool_size=(2,2), padding='same', name="pool2")(relu2)

# 第3层卷积，将第二层的maxpoling得出来的值作为第三层的输入，在第三层卷积的时候我们又扩大了卷积核的数量
conv3 = Conv2D(64, (3, 3), name = "conv3")(pool2)
relu3 = Activation('relu', name="relu3")(conv3)
# 做MaxPooling 设置池化大小为2*2，表示每一个2*2的像素都会变成一个像素，这样整个图像的参数就会变少，得到池化后的值为pool3
pool3 = MaxPooling2D(pool_size=(2,2), padding='same', name="pool3")(relu3)

# 将 Pooled feature map 摊平后输入全连接网络
x = Flatten()(pool3)

# Dropout，防止过拟合
x = Dropout(0.25)(x)

# 4个全连接层分别做10分类，就是说每一个字符有十个类别，分别对应4个字符。此时x就变成了一个Captcha也就是我们的验证码
x = [Dense(10, activation='softmax', name='fc%d'%(i+1))(x) for i in range(4)]

# 4个字符向量拼接在一起，与标签向量形式一致，作为模型输出。就对应我们四个one-hot拼接起来的向量
outs = Concatenate()(x)

# 定义模型的输入与输出，inputs就是形状为[None,60，160，1]这样形状的图像。outputs就是输出长位40的向量
model = Model(inputs=inputs, outputs=outs)
# 对模型进行编译，输入三个参数 优化器，损失函数，评价指标(指定为准确率)
model.compile(optimizer=OPT, loss=LOSS, metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [27]:
print(X_train[0])

[[[0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883922]
  [0.97883

## 查看模型摘要

In [ ]:
model.summary()

## 模型可视化

In [25]:
# model 我们的模型， ti_file 存储模型的路径 
plot_model(model, to_file=MODEL_VIS_FILE, show_shapes=True)

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

## 训练模型

### history就是我们训练模型的输出记录
#### 可以用这些数据画折线图，直观看到：

__模型收敛的速度（斜率）           模型是否已经收敛（稳定性）           模型是否过拟合（验证数据集）__


In [26]:
# history就是我们训练模型的输出记录
history = model.fit(X_train,
                    Y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=2,
                    validation_data=(X_test, Y_test))

Instructions for updating:
Use tf.cast instead.
Train on 1211 samples, validate on 954 samples
Epoch 1/2
 - 40s - loss: 0.3252 - acc: 0.8963 - val_loss: 0.3731 - val_acc: 0.9000
Epoch 2/2
 - 38s - loss: 0.2966 - acc: 0.9000 - val_loss: 0.4196 - val_acc: 0.9000


## 预测样例

In [28]:
# 实际的验证码
print(vec2text(Y_test[9]))

0111


In [30]:
# 打开图片进行验证
img = rgb2gray(np.array(Image.open('3935.png')))
img=img/255
yy = model.predict(img.reshape(1, 60, 160, 1))

In [31]:
# 预测出来的验证码保存到yy变量中
yy = model.predict(X_test[9].reshape(1, 60, 160, 1))
print(yy)

[[3.14430773e-01 2.95201272e-01 3.87412786e-01 1.10699190e-03
  3.07912560e-04 3.17515922e-04 2.72155012e-04 3.36285535e-04
  2.95922538e-04 3.18263512e-04 8.66983831e-02 9.00086686e-02
  1.29232615e-01 1.04191884e-01 7.28161857e-02 9.86915529e-02
  9.22656581e-02 8.81439969e-02 8.59639794e-02 1.51987001e-01
  7.84292817e-02 1.41196758e-01 9.87205505e-02 7.86532238e-02
  7.66169876e-02 7.36141503e-02 1.33546963e-01 9.79153365e-02
  9.85080227e-02 1.22798771e-01 8.70112553e-02 6.31042570e-02
  1.01753846e-01 9.58472490e-02 1.02666937e-01 8.24608728e-02
  1.28385946e-01 1.23980545e-01 1.13501750e-01 1.01287425e-01]]


In [32]:
# 输出yy 可以看到和我们实际的验证码不一致，预测错误
print(vec2text(yy))

2916


## 保存模型

In [ ]:
if not gfile.Exists(MODEL_DIR):
    gfile.MakeDirs(MODEL_DIR)

model.save(MODEL_FILE)
print('Saved trained model at %s ' % MODEL_FILE)

## 保存训练过程记录

In [ ]:
# history,history 就相当于一个字典 history.history['acc']查看key为acc的value 也就是查看我们的精确值
history.history['acc']

In [ ]:
# 查看字典里面的元素
history.history.keys()

In [ ]:
if gfile.Exists(HISTORY_DIR) == False:
    gfile.MakeDirs(HISTORY_DIR)

with open(HISTORY_FILE, 'wb') as f:
    # 使用 pickle.dump 对字典进行一个序列化，最后保存在f文件中
    pickle.dump(history.history, f)

In [ ]:
print(HISTORY_FILE)